## LIBRARIES

In [ ]:
!pip install pysrt
!pip install keras_tuner

In [ ]:
import random , pandas as pd , numpy as np , re , pysrt , tensorflow as tf , gensim
from pathlib import Path
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential ,load_model
from tensorflow.keras.layers import  LSTM, Dense , Activation , Embedding, MaxPool1D ,GlobalMaxPool1D , Conv1D , BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer

## LOADING DATA

In [ ]:

folder_path = Path("/content/")
files = list(folder_path.glob('*'))
text = ''

for file_ in files:
    if str(file_)[-3:] == 'srt' :
        print(file_)
        subs = pysrt.open(file_)
        text_ = '\n'.join(sub.text for sub in subs)
        text = ' '.join([text,text_])

print(text)

print(type(text))

## PREPROCESSING

In [ ]:
# Ran the loop twice beacuse treating subtitles of one film creates issues in other film subtiles ,so running twice helps.

for i in range(2):
    text = text.lower().replace('</i>', '').replace('♪', '').replace('<i>', '').replace('...','').replace(',',' , ' ).replace('\'',"'").replace('\n',' ').replace('  ',' ').replace(' - ' , ' ').replace("--","").replace('"','').replace(".","").replace('wwwfacebookcom/englishlolchannel/','')
    pattern1 = r'\[.*?\]|\?'
    result = re.sub(pattern1, '', text)
    pattern2 = r'{.}'
    result = re.sub(pattern2, '', result)
    pattern3 = r"(\w+)'"
    result = re.sub(pattern3,'g',result)
    pattern4 = r'<.*?>'
    result = re.sub(pattern4,"",result)
    pattern5 = r'(\d+)[ ,]+(\d+)'
    result = re.sub(pattern5,"",result)
print(result)


In [ ]:
#saving the processed text to have a good look

# with open("C:/Users/Lenovo/Desktop/next work/lafs.txt", "w") as file:
#     file.write(result)

## tokenizing the texts

In [ ]:
# tokenization
tokenizer = RegexpTokenizer(r"\S+")
tokens = tokenizer.tokenize(result.lower())
print(tokens[:15])

print(len(tokens))
print(len(np.unique(tokens)))

In [ ]:
unique_tokens = np.unique(tokens)
unique_tokens_index = {token : idx for idx , token in enumerate(unique_tokens)}
print(unique_tokens_index)

## creating the input and output words for the model similar to x and y

In [ ]:
# preparing the data
nwords = 10
input_words = []
output_words = []

for i in range(len(tokens)-nwords):
    input_words.append(tokens[i : i+nwords])
    output_words.append(tokens[i+nwords])

print(input_words[65])
print(output_words[65])

print(len(input_words))
print(len(output_words))

In [ ]:
print(input_words[0])

## using gensim to create vectors for each word

In [ ]:
dim = 100
wrd2vc = gensim.models.Word2Vec(sentences = input_words , vector_size = dim , window = 10 ,  min_count = 1 )

In [ ]:
wrd2vc.wv.most_similar('leave')

## coming up with X and Y in form of arrays
## Replacing all the words with their respective token numbers fro X
## creating a sparse categorical for Y

In [ ]:
x = np.array(input_words)

y = np.zeros((len(output_words),len(unique_tokens)))

for i in range(len(input_words)):
    for j in range(len(input_words[i])):
        x[i,j] = unique_tokens_index[x[i,j]]
    y[i,unique_tokens_index[output_words[i]]] = 1

In [ ]:
print(x.shape)
print(y.shape)

## creating a weight matrix for embedding layer

In [ ]:
def wt_matrix(model):
  weight_matrix = np.zeros((len(unique_tokens) , dim))

  for word_ , ind_ in unique_tokens_index.items():
    weight_matrix[ind_] = model.wv[word_]

  return weight_matrix

In [ ]:
embedded_vectors = wt_matrix(wrd2vc)

In [ ]:
embedded_vectors.shape

## model
## 2 models but doing the same , 1st model is the elaborated version of model 2

In [ ]:
# def tuner_model(hp):

#     model = Sequential()
#     model.add(Embedding( len(unique_tokens) , output_dim = dim , weights = [embedded_vectors] , trainable = False))


#     counter_ = 0

#     for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

#         if counter_ < 10:
#             model.add(
#                        LSTM(
#                             hp.Int('units'+str(i),min_value = 8 , max_value = 256 , step = 8),
#                             activation= hp.Choice('activation'+str(i) , values = ['relu','tanh']),
#                             return_sequences = True
#                             )
#                      )

#         else :
#             model.add(
#                        LSTM(
#                             hp.Int('units'+str(i),min_value = 8 , max_value = 256 , step = 8),
#                             activation= hp.Choice('activation'+str(i) , values = ['relu','tanh']),
#                             return_sequences = False
#                             )
#                      )

#         counter_ += 1


#         # Apply Batch Normalization if chosen by the tuner
#         if hp.Boolean('batch_norm'+str(i)):
#             model.add(BatchNormalization())

#         # Apply Max Pooling if chosen by the tuner
#         if hp.Boolean('max_pooling'+str(i)):
#             model.add(GlobalMaxPool1D())


#     model.add(Dense(len(unique_tokens) , activation = 'softmax'))


#     model.compile( optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adagrad' , 'adadelta', 'nadam']),
#                    loss = 'categorical_crossentropy',
#                    metrics =['accuracy']
#                    )

#     return model

In [ ]:
def tuner_model(hp):

    model = Sequential()
    model.add(Embedding( len(unique_tokens) , output_dim = dim , weights = [embedded_vectors] , trainable = False))

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

        units = hp.Int('units_'+str(i),min_value = 8 , max_value = 256 , step = 8)
        activation = hp.Choice('activation_'+str(i), values = ['relu','tanh'])
        return_sequences = True if i < 10 else False

        model.add(LSTM(units, activation=activation, return_sequences=return_sequences))

        # using ' IF '   ,  if its prooven beneficial keras_tuner keeps it
        if hp.Boolean('batch_norm'+str(i)):
            model.add(BatchNormalization())

        # Apply Max Pooling if chosen by the tuner
        if hp.Boolean('max_pooling'+str(i)):
            model.add(GlobalMaxPool1D())

    model.add(Dense(len(unique_tokens) , activation = 'softmax'))

    optimizer = hp.Choice('optimizer', values = ['adam','sgd','rmsprop','adagrad' , 'adadelta', 'nadam'])

    model.compile(optimizer = optimizer,
                  loss = 'categorical_crossentropy',
                  metrics =['accuracy']
                  )

    return model


## now tuning the model using keras tuner

In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(tuner_model,objective = 'val_accuracy' , max_epochs =  5 , factor = 3 , directory = 'keras_tuner',project_name=  'tuner_model')

In [ ]:

earlystop = EarlyStopping(monitor = 'val_loss',patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience = 3)

In [ ]:
# lets say the model tuning encounters some error at 2nd epoch , then we need to remove the model tuning that happend till now

# !rm -r /content/keras_tuner/tuner_model

In [ ]:
# tuner_.search(x,y,epochs = 5 , validation_split=0.2 ,callbacks = [earlystop,reduce_lr])

# some times it happens that the tuning runs for more than 5 epochs for some reasons , so to avoid that we use for loop

max_trials = 5

for i in range(max_trials):
    tuner.search(x, y, epochs=5, validation_split=0.2, callbacks=[earlystop, reduce_lr])


In [ ]:
model1 = tuner.hypermodel.build(tuner.get_best_hyperparameters(num_trials=1)[0])
model1.summary()

In [ ]:
# saving the best tuned model in case there is any issue like internet disconnection then we can resume from here.

# best_model = tuner.get_best_models(num_models=1)[0]
# best_model.save('best_model_search.h5')

In [ ]:
# downloading the saved best tuned model

# from google.colab import files

# files.download('best_model_search.h5')


## model fitting

In [ ]:
model1.fit(x,y,epochs = 100 , validation_split=0.2 ,callbacks = [earlystop,reduce_lr], batch_size=128,shuffle=True)

In [ ]:
# model1.save('my_model.h5')

# from google.colab import files
# files.download('my_model.h5')

In [ ]:
# from tensorflow.keras.models import load_model

# model = load_model('/content/best_model.h5')


## making predictions

In [ ]:

def pred(input_ ,nbest):
    input_ = input_.lower()
    x = np.zeros((1,nwords,len(unique_tokens)))
    for i,word in enumerate(input_.split()):
        x[0,i,unique_tokens_index[word]] = 1

    predict_ = model.predict(x)[0]
    return np.argpartition(predict_ , -nbest)[-nbest:]


In [ ]:
pp = pred('probably chose to be four minutes late',5)
print([unique_tokens[i] for i in pp])

## this gives the predictoins for next 5 words


In [ ]:
## making predictions for next 100 words


def generate_text(input_text , text_length , creativity = 3):
    word_seq = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_seq =  " ".join(tokenizer.tokenize(" ".join(word_seq).lower())[current:current+nwords])
        choice = unique_tokens[random.choice(pred(sub_seq,creativity))]
        word_seq.append(choice)
        current+=1
    return " ".join(word_seq)


In [ ]:

generate_text('probably chose to be four minutes late',100,5)
